<a href="https://colab.research.google.com/github/awadell1/MLOnAGooseberryBush/blob/master/src/LSTM_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import text_to_word_sequence
from tqdm.notebook import tqdm
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
SEED = 42 #setting the seed for reproducebility
AUTOTUNE = tf.data.experimental.AUTOTUNE #currently not entirely sure what this can do

In [ ]:
import sys, os
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)
!pip install --target=$nb_path aion


  Created wheel for aion: filename=Aion-0.1-cp36-none-any.whl size=942 sha256=b0ceb37e6c6b5e8a43efad7a6bfe08af710d47c56289fdd27304f8299294a71c
  Stored in directory: /root/.cache/pip/wheels/07/57/2b/8d23b69d0bfbba62ef07de99bcccdad45ee152aed5792978eb
Successfully built aion


In [ ]:
!pip install symspellpy

     |████████████████████████████████| 2.6MB 13.6MB/s 


In [ ]:
word_df = pd.read_csv('/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/word_counts_newest.csv',index_col='Unnamed: 0')
appears_more_than_once=word_df[word_df['0']>1]
dict_appears_more_than_once=appears_more_than_once.to_dict()['0']
appears_more_than_once=appears_more_than_once.astype(np.int64)

In [ ]:
from collections import Counter
from aion.util.spell_check import SpellCorrector
#make the correction results more accessible
corpus=Counter(dict_appears_more_than_once)
spell_corrector = SpellCorrector(dictionary=corpus)
appears_once=word_df[word_df['0']<=1]
appears_once=appears_once.astype(np.int64).reset_index()[['index']]
appears_once.columns=['original']
def correction(word):
  return spell_corrector.correction(word)
tqdm.pandas()
appears_once['corrected'] = appears_once['original'].progress_apply(lambda x: correction(x))

In [ ]:
corrected_ls=list(appears_once.to_dict()['corrected'].values())
orig_ls=list(appears_once.to_dict()['original'].values())
fix_box=dict(zip(orig_ls,corrected_ls))

In [ ]:
# sentence=sentence_printout(path='/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/review_cleaned.txt')
# raw_df=pd.read_csv('/content/drive/MyDrive/24787 Group Project  /Data/winemag-data-130k-v2.csv',index_col=0)
# raw_label=raw_df[['points']]
raw_df=pd.read_csv('/content/drive/MyDrive/24787 Group Project  /Data/winemag-data-130k-v2.csv',index_col=0)
raw_data=raw_df[['description']]
raw_label=raw_df[['points']]

In [ ]:
def review_cleanup(revised_dict,x):
  review_ls=text_to_word_sequence(x[0],filters='­­"…–â€˜‘’—!#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r“”')
  revised=[revised_dict.get(word,word) for word in review_ls]
  return revised

In [ ]:
raw_data['corrected_once']=raw_data.progress_apply(lambda x: review_cleanup(fix_box,x),axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
save_path='/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/vectors100_cleaned.tsv'
word2vec_map = pd.read_csv(save_path,sep='\t',header=None)
print(word2vec_map.shape)
word2vec_map.head(2)

(24421, 100)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.173625,-0.044444,0.113342,0.083261,-0.060938,-0.132793,0.328555,0.224562,0.185575,-0.161447,0.228990,0.165603,-0.005079,0.304789,0.265582,0.501026,-0.119153,0.074965,0.249299,-0.024614,-0.111520,0.016991,0.143916,-0.038430,-0.046623,0.265628,0.312073,0.01483,-0.174740,0.200961,-0.185053,-0.179840,-0.105220,-0.306932,0.099202,0.062982,0.12285,-0.111038,0.162552,0.176298,...,0.126548,0.117353,-0.211523,-0.150847,-0.281774,0.004737,-0.070999,0.263961,0.068182,-0.113628,0.159668,-0.199827,-0.104189,-0.080333,0.163228,-0.552201,0.160589,-0.095748,-0.288310,0.134207,-0.053729,0.315701,0.150047,0.205081,0.087938,-0.296919,0.103946,-0.096799,0.398342,-0.306436,0.311119,-0.080264,-0.274337,0.501867,-0.194469,-0.003606,-0.163824,0.150878,-0.224293,0.223331
1,-0.257154,-0.363296,0.020309,-0.499152,0.189040,-0.371158,-0.573646,0.401893,0.218572,-0.122283,-0.148845,0.106698,0.288068,-0.026458,-0.341448,0.439929,0.211149,-0.121051,-0.047289,0.362859,-0.040821,-0.004571,-0.202173,-0.350804,0.290275,0.033538,0.206428,0.53440,-0.231279,-0.263137,-0.338335,0.222076,-0.065504,0.233159,0.056918,-0.477155,0.03380,-0.363502,0.201924,0.407232,...,-0.189471,-0.186601,0.205003,-0.259861,0.434576,0.563895,0.254541,-0.000926,-0.119696,-0.344114,-0.356312,-0.849274,-0.449690,-0.568843,-0.186584,-0.215586,0.317289,0.548595,0.634312,0.050545,-0.054913,-0.022056,0.039310,-0.193269,0.528237,-0.581558,-0.074250,-0.302481,-0.007905,-0.248025,-0.196104,-0.541709,-0.118898,0.582164,-0.338514,0.133960,0.490275,0.145898,-0.246337,-0.062087


In [ ]:
def sentence_printout(path):
    #just to demonstrate the txt file (the first ten sentence)
    sentence=[]
    with open(path) as f: 
        lines = f.read().splitlines()
    sentence = []
    for line in lines:
        filter(None, line.split('\t'))
        sentence.append(list(filter(None, line.split('\t'))))
    f.close()
    return sentence

In [ ]:
I2W_df=pd.read_csv('/content/drive/My Drive/24787 Group Project  /Data/for_final_submission/Dictionary_cleaned.csv')

I2W=I2W_df.to_dict(orient='dict')['Vocab']
I2W[0] = ''
W2I = {v: k for k, v in I2W.items()}

In [ ]:
def sentence_to_indices(data,w2ind):
    sentences_num=[]
    for row in tqdm(data):
        temp_mem=[w2ind[word] for word in row]
        sentences_num.append(temp_mem)
    return sentences_num

In [ ]:
# sentences_num = sentence_to_indices(data=sentence,w2ind=W2I)
sentences_num = sentence_to_indices(data=raw_data['corrected_once'],w2ind=W2I)

In [ ]:
for seq in sentences_num[:2]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[15, 874, 202, 13, 1981, 3598, 1, 88, 105, 2, 16, 909, 1031, 928, 308, 13741, 48, 60, 1, 88, 421, 142, 360, 18] 
 => 
 ['aromas', 'include', 'tropical', 'fruit', 'broom', 'brimstone', 'and', 'dried', 'herb', 'the', 'palate', "isn't", 'overly', 'expressive', 'offering', 'unripened', 'apple', 'citrus', 'and', 'dried', 'sage', 'alongside', 'brisk', 'acidity']
[6, 7, 27, 1, 75, 3, 8, 17, 7, 99, 61, 123, 144, 78, 22, 28, 1063, 90, 5, 72, 30, 43, 49, 1, 3745, 5, 18, 24, 379, 702, 291, 12, 87, 684, 96, 439, 20, 340] 
 => 
 ['this', 'is', 'ripe', 'and', 'fruity', 'a', 'wine', 'that', 'is', 'smooth', 'while', 'still', 'structured', 'firm', 'tannins', 'are', 'filled', 'out', 'with', 'juicy', 'red', 'berry', 'fruits', 'and', 'freshened', 'with', 'acidity', "it's", 'already', 'drinkable', 'although', 'it', 'will', 'certainly', 'be', 'better', 'from', '2016']


In [ ]:
reviews_num = sentences_num
max_review_length = max([len(seq) for seq in reviews_num])
input_sequences = np.array(pad_sequences(reviews_num, maxlen=max_review_length, padding='pre'))

In [ ]:
for seq in input_sequences[:1]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0    15   874   202    13  1981  3598     1    88
   105     2    16   909  1031   928   308 13741    48    60     1    88
   421   142   360    18] 
 => 
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [ ]:
input_sequences.shape

(129971, 136)

In [ ]:
#for some reason the input_sequences have a shape of 129973
#while the total example number is 129971
#i will randomly drop 2 input sequences since the fraction is very low
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
input_sequences=input_sequences[targ_ind[2:]]

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, index_to_word):
    """
    Creates a Keras Embedding() layer
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(index_to_word)+1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map.shape[1]      # define dimensionality of your word vectors (= 256)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for index, word in tqdm(index_to_word.items()): #skip the first item "<padding>"
        emb_matrix[index,:]=word_to_vec_map.iloc[index-1]
      # word_check = dict_pre.get(word)
      # if word_check is not None:
      #     emb_matrix[index,:] = word_to_vec_map[word].to_numpy().reshape(emb_dim,)
      # else:
          
      #     emb_matrix[index,:] = average_average_calculator(index, word_to_vec_map,sequence,dict_pre)

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim,trainable=True,mask_zero=True) 
    ##TODO:is this maskz_zero correct here?
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
I2W_nopad=I2W.copy()
embedding_layer = pretrained_embedding_layer(word2vec_map, I2W_nopad)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])


weights[0][1][3] = 0.083260976


In [ ]:
def WineReviewMachine_V2(input_shape, word_to_vec_map, index_to_word, num_class):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32') #the max length 
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, index_to_word)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    # max_batch_size=1024
    # partitions = tf.range(max_batch_size)
    # num_partitions = max_batch_size
    # x_unpacked = tf.dynamic_partition(embeddings, partitions, num_partitions, name='dynamic_unstack')
    # #x_unpacked = tf.unstack(embeddings) # defaults to axis 0, returns a list of tensors
    # mini_batched_processed=[]
    # for x in x_unpacked:
    #    partitions = tf.range(136)
    #    num_partitions = 136
    #    t_ls = tf.dynamic_partition(x, partitions, num_partitions, name='dynamic_unstack') 
    #    average_embedding = tf.reduce_mean(x, 0)
    #    processed=[]
    #    for t in t_ls:
    #       if tf.math.count_nonzero(t).numpy()[0]==0:
    #          results_tensor=average_embedding
    #       else:
    #          results_tensor=t
    #       processed.append(result_tensor)
    #    batched_embeddings=tf.concat(processed,0)
    #    mini_batched_embeddings.append(batched_embeddings)
    # embeddings=tf.concat(mini_batched_embeddings,0)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(num_class)(X)
    # Add a softmax activation
    X = Activation("softmax")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
num_class=3
model = WineReviewMachine_V2((max_review_length,), word2vec_map, I2W, num_class)
model.summary()


Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 136, 100)          2442300   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 136, 512)          731136    
_________________________________________________________________
dropout_8 (Dropout)          (None, 136, 512)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)               

In [ ]:
y_label=raw_label.values
y_label_3levels = []
for i,y0 in enumerate(y_label):
  temp = 0
  if y0 <87:
    temp = int(0)
  elif y0>=87 and y0<93:
     temp = int(1.0)
  elif y0>=93: temp = int(2.0)
  y_label_3levels.append(temp)

In [ ]:
indices = y_label_3levels
depth = max(indices)+1
y_label_one_hot=tf.one_hot(indices, depth)

In [ ]:
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_label_one_hot[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_label_one_hot[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset_train = tf.data.Dataset.from_tensor_slices((x_train_vec, y_train_label))
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset_test = tf.data.Dataset.from_tensor_slices((x_test_vec, y_test_label))
dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)
print(dataset_test)

<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset_train)
print(dataset_test)

<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#trained 25 epochs
model.fit(dataset_train,epochs=15,verbose=1,validation_data=dataset_test)

Epoch 1/15
117/117 [==============================] - 62s 532ms/step - loss: 0.3901 - accuracy: 0.8323 - val_loss: 0.4680 - val_accuracy: 0.7994
Epoch 2/15
117/117 [==============================] - 63s 535ms/step - loss: 0.3646 - accuracy: 0.8441 - val_loss: 0.4820 - val_accuracy: 0.7996
Epoch 3/15
117/117 [==============================] - 62s 534ms/step - loss: 0.3399 - accuracy: 0.8567 - val_loss: 0.5118 - val_accuracy: 0.7990
Epoch 4/15
117/117 [==============================] - 62s 529ms/step - loss: 0.3227 - accuracy: 0.8644 - val_loss: 0.5197 - val_accuracy: 0.7972
Epoch 5/15
117/117 [==============================] - 62s 527ms/step - loss: 0.3145 - accuracy: 0.8686 - val_loss: 0.5178 - val_accuracy: 0.7945
Epoch 6/15
117/117 [==============================] - 62s 527ms/step - loss: 0.2919 - accuracy: 0.8790 - val_loss: 0.5737 - val_accuracy: 0.7871
Epoch 7/15
117/117 [==============================] - 62s 528ms/step - loss: 0.2738 - accuracy: 0.8883 - val_loss: 0.6455 - val_ac

In [ ]:
def WineReviewMachine_V3(input_shape, word_to_vec_map, index_to_word):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32') #the max length 
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, index_to_word)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    # max_batch_size=1024
    # partitions = tf.range(max_batch_size)
    # num_partitions = max_batch_size
    # x_unpacked = tf.dynamic_partition(embeddings, partitions, num_partitions, name='dynamic_unstack')
    # #x_unpacked = tf.unstack(embeddings) # defaults to axis 0, returns a list of tensors
    # mini_batched_processed=[]
    # for x in x_unpacked:
    #    partitions = tf.range(136)
    #    num_partitions = 136
    #    t_ls = tf.dynamic_partition(x, partitions, num_partitions, name='dynamic_unstack') 
    #    average_embedding = tf.reduce_mean(x, 0)
    #    processed=[]
    #    for t in t_ls:
    #       if tf.math.count_nonzero(t).numpy()[0]==0:
    #          results_tensor=average_embedding
    #       else:
    #          results_tensor=t
    #       processed.append(result_tensor)
    #    batched_embeddings=tf.concat(processed,0)
    #    mini_batched_embeddings.append(batched_embeddings)
    # embeddings=tf.concat(mini_batched_embeddings,0)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(20)(X)
    X = Dense(1)(X)
    # Add a softmax activation
    X = Activation("linear")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
y_test=raw_label.values-80
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_test[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_test[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
model_linear100 = WineReviewMachine_V3((max_review_length,), word2vec_map, I2W)
model_linear100.summary()


Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 136, 100)          2442300   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 136, 512)          731136    
_________________________________________________________________
dropout_10 (Dropout)         (None, 136, 512)          0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 512)               1574912   
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 20)             

In [ ]:
model_linear100.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=[tf.keras.losses.MeanSquaredError()])

In [ ]:
#trained 9 epochs
model_linear100.fit(dataset_train,epochs=15,verbose=1,validation_data=dataset_test)

start 300D

In [ ]:
save_path='/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/vectors300.tsv'
word2vec_map = pd.read_csv(save_path,sep='\t',header=None)
print(word2vec_map.shape)
word2vec_map.head(2)

(33167, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.072741,0.177086,0.046955,-0.161159,-0.262453,-0.022582,-0.175011,0.193300,0.081327,0.513394,0.486482,0.270007,-0.055469,-0.017736,0.233057,-0.339189,0.080459,-0.211314,0.230841,-0.092777,-0.354089,0.353036,-0.032889,-0.039103,-0.100094,-0.262138,-0.219273,-0.020130,-0.142205,0.016919,-0.149281,-0.289330,-0.298905,-0.299549,-0.046952,0.050852,-0.162308,-0.301594,-0.210345,-0.224959,...,0.269495,0.561228,0.110157,0.371961,-0.091705,-0.042529,-0.270073,-0.233123,0.027484,-0.057847,0.025248,0.312116,0.155597,0.338420,0.188724,0.064901,0.057733,0.408761,0.200816,-0.221229,-0.000132,0.308902,-0.411560,0.268989,-0.036705,0.528598,-0.220828,0.026004,-0.039186,-0.078684,-0.030462,0.147154,-0.264258,0.445950,-0.252144,-0.463734,-0.086174,0.195977,0.044055,-0.141106
1,-0.162451,0.056447,0.074096,0.247125,0.275788,-0.075669,-0.405454,0.067573,-0.296444,-0.120678,0.676957,-0.044911,0.046235,0.292248,0.051215,0.366189,-0.120822,0.290646,-0.205931,-0.146317,0.129239,0.328307,-0.261547,0.371859,0.095685,-0.247538,-0.391735,0.014415,0.118940,-0.245449,0.211864,0.195619,0.209814,0.063449,-0.125104,-0.066620,0.020668,-0.013998,-0.467575,0.248410,...,0.273146,0.058361,-0.292375,-0.193279,-0.226322,0.058354,-0.225004,-0.123047,0.010058,-0.657200,-0.042546,0.359351,-0.180970,0.175821,0.042868,0.461206,-0.026787,-0.333164,0.025929,0.368363,-0.224638,-0.160631,0.348195,0.179474,0.058651,0.425275,-0.291038,0.428397,-0.703248,0.350351,0.058166,-0.252873,-0.376265,0.005129,-0.359104,0.110904,0.242587,-0.066842,0.333056,0.044874


In [ ]:
I2W_nopad=I2W.copy()
embedding_layer300 = pretrained_embedding_layer(word2vec_map, I2W_nopad)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])


weights[0][1][3] = 0.083260976


In [ ]:
y_label=raw_label.values
y_label_3levels = []
for i,y0 in enumerate(y_label):
  temp = 0
  if y0 <87:
    temp = int(0)
  elif y0>=87 and y0<93:
     temp = int(1.0)
  elif y0>=93: temp = int(2.0)
  y_label_3levels.append(temp)

In [ ]:
indices = y_label_3levels
depth = max(indices)+1
y_label_one_hot=tf.one_hot(indices, depth)

In [ ]:
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_label_one_hot[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_label_one_hot[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset_train = tf.data.Dataset.from_tensor_slices((x_train_vec, y_train_label))
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset_test = tf.data.Dataset.from_tensor_slices((x_test_vec, y_test_label))
dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)
print(dataset_test)

<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset_train)
print(dataset_test)

<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
num_class=3
model300 = WineReviewMachine_V2((max_review_length,), word2vec_map, I2W_nopad, num_class)
model300.summary()


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 136, 300)          7326900   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 136, 512)          1140736   
_________________________________________________________________
dropout_4 (Dropout)          (None, 136, 512)          0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)               

In [ ]:
model300.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model300.fit(dataset_train,epochs=15,verbose=1,validation_data=dataset_test)

117/117 [==============================] - 74s 631ms/step - loss: 0.8760 - accuracy: 0.6337 - val_loss: 0.8574 - val_accuracy: 0.6421
Epoch 2/10
117/117 [==============================] - 71s 604ms/step - loss: 0.8676 - accuracy: 0.6363 - val_loss: 0.8582 - val_accuracy: 0.6421
Epoch 3/10
117/117 [==============================] - 71s 606ms/step - loss: 0.8617 - accuracy: 0.6363 - val_loss: 0.8619 - val_accuracy: 0.6421
Epoch 4/10
117/117 [==============================] - 71s 607ms/step - loss: 0.8460 - accuracy: 0.6385 - val_loss: 0.8763 - val_accuracy: 0.6402
Epoch 5/10
117/117 [==============================] - 70s 601ms/step - loss: 0.8117 - accuracy: 0.6515 - val_loss: 0.9151 - val_accuracy: 0.6255
Epoch 6/10
117/117 [==============================] - 70s 602ms/step - loss: 0.7617 - accuracy: 0.6732 - val_loss: 0.9612 - val_accuracy: 0.6144
Epoch 7/10
117/117 [==============================] - 70s 600ms/step - loss: 0.7119 - accuracy: 0.6955 - val_loss: 0.9965 - val_accuracy: 0.6

In [ ]:
y_test=raw_label.values-80
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_test[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_test[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
model_linear300 = WineReviewMachine_V3((max_review_length,), word2vec_map, I2W_nopad)
model_linear300.summary()


Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 136, 300)          7326900   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 136, 512)          1140736   
_________________________________________________________________
dropout_6 (Dropout)          (None, 136, 512)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)              

In [ ]:
model_linear300.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=[tf.keras.losses.MeanSquaredError()])
model_linear300.fit(dataset_train,epochs=10,verbose=1,validation_data=dataset_test)

Epoch 1/10
117/117 [==============================] - 74s 629ms/step - loss: 0.2418 - mean_squared_error: 0.2418 - val_loss: 0.2224 - val_mean_squared_error: 0.2224
Epoch 2/10
117/117 [==============================] - 71s 610ms/step - loss: 0.2240 - mean_squared_error: 0.2240 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 3/10
117/117 [==============================] - 71s 604ms/step - loss: 0.2232 - mean_squared_error: 0.2232 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 4/10
117/117 [==============================] - 70s 600ms/step - loss: 0.2228 - mean_squared_error: 0.2228 - val_loss: 0.2222 - val_mean_squared_error: 0.2222
Epoch 5/10
117/117 [==============================] - 71s 607ms/step - loss: 0.2226 - mean_squared_error: 0.2226 - val_loss: 0.2222 - val_mean_squared_error: 0.2222
Epoch 6/10
117/117 [==============================] - 71s 607ms/step - loss: 0.2225 - mean_squared_error: 0.2225 - val_loss: 0.2222 - val_mean_squared_error: 0.2222
Epoch 7/10